In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import random
from random import choice,randrange
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import math
import time
from torch_geometric.nn import GCNConv,GraphConv,GATConv,SAGEConv

random_seed=280085

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 5.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 5.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 2.4 MB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 750 kB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███

In [4]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/My Drive/asproject


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1Z1Bpxa6AiU_gI6sC4xffcphQt3aexiNK/asproject


In [5]:
from utils import *  #In this files are reported the most useful functions

In [6]:
X = torch.load('instance').T      #Instance matrix
A = torch.load('adjacencyCOO')  #Normal adjacency matrix format is obtained with torch.load('adjacency')


In [7]:
num2artist = load_data('dizofartist.json')
artist2num = {num2artist[key]:key for key in num2artist}

## Import the data with Torch geometric:


In [10]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import train_test_split_edges

data = Data(x=X, edge_index = A)
#loader = DataLoader([data], batch_size=512, shuffle=True)

In [13]:
data.edge_index

tensor([[    0,     0,     0,  ..., 11260, 11260, 11260],
        [ 1386,  5175,  6682,  ...,  9153, 10229, 11180]])

## Define the architectures

* GraphSage-based architecture is the one defined in the original paper [Artist similarity with Graph Neural Networks](https://arxiv.org/pdf/2107.14541.pdf).  
  1. SAGEConv(2613,256)
  2. SAGEConv(256,256) *
  3. SAGEConv(256,256) *
  4. Linear(256,100)  
  
The second and third layer are optionals based on the desired configuration.

* The GAT-based architectures are the one used for our own experiments, they are defined as follows:  

GAT1: 
  1. GATConv(2613, 256)    *Multi-head attention mechanism
  2. GATConv(256 * 4, 256) *Multi-head attention mechanism
  3. Linear(256 * 4, 256)  
  4. Linear(256, 256)
  5. Linear(256, 256)  

GAT2:  
  1. Linear(2613, 256)
  2. Linear(256,256)
  3. GATConv(2613, 256)    *Multi-head attention mechanism
  4. GATConv(256 * 4, 256) *Multi-head attention mechanism
  5. Linear(256 * 4, 256) 

In [37]:
class GraphSage(nn.Module):
  
  def __init__(self, n_layers):
    super(GraphSage, self).__init__()
    self.nl = n_layers
    self.GS1 = SAGEConv(2613,256, aggr = 'mean', normalize = True, bias = True)
    if self.nl >1 and self.nl<=3:
      self.GS2 = SAGEConv(256,256, aggr = 'mean', normalize = True, bias = True)
    if self.nl >2:
      self.GS3 = SAGEConv(256,256, aggr = 'mean', normalize = True, bias = True)

    self.linear = nn.Linear(256,100)
  
  def forward(self, x, edges):

    x = F.elu(self.GS1(x,edges))
    if self.nl >1 and self.nl<=3:
      x = F.elu(self.GS2(x,edges))
    if self.nl >2:
      x = F.elu(self.GS3(x,edges))

    x = F.elu(self.linear(x))

    return x

class GAT1(nn.Module):
  
  def __init__(self):
    super(GAT1, self).__init__()

    n_heads = 4

    self.GAT1_1 = GATConv(2613,256, heads = 4, bias = True)
    self.GAT1_2 = GATConv(n_heads*256,256, heads = 4, bias = True)

    self.linear1 = nn.Linear(n_heads*256,256)
    self.linear2 = nn.Linear(256,256)
    self.linear3 = nn.Linear(256,256)

  
  def forward(self, x, edges):

    x = F.elu(self.GAT1_1(x,edges))
    x = F.elu(self.GAT1_2(x,edges))

    x = F.elu(self.linear1(x))
    x = F.elu(self.linear2(x))
    x = F.elu(self.linear3(x))

    return x

class GAT2(nn.Module):
  
  def __init__(self):
    super(GAT2, self).__init__()

    n_heads = 4

    self.GAT2_1 = GATConv(256,256, heads = 4, bias = True)
    self.GAT2_2 = GATConv(n_heads*256,256, heads = 4, bias = True)

    self.linear1 = nn.Linear(2613,256)
    self.linear2 = nn.Linear(256,256)
    self.linear3 = nn.Linear(n_heads*256,256)


  
  def forward(self, x, edges):

    x = F.elu(self.linear1(x))
    x = F.elu(self.linear2(x))
    
    x = F.elu(self.GAT2_1(x,edges))
    x = F.elu(self.GAT2_2(x,edges))
    x = F.elu(self.linear3(x))


    return x

n_layers = 3
SageModel = GraphSage(n_layers = n_layers)
Gat1Model = GAT1()
Gat2Model = GAT2()
